<a name="cell-solving"></a>

### 4. [Direct Methods](#cell-sovling-Axb) /  [Algorithmic Speed](#cell-sovling-Axb)


0. [[OPTIONAL] Finding $X\hat \beta \approx y$ by solving for $\hat \beta$ in $\overset{A\, x}{X^T\!X \hat \beta} = \overset{b}{X^t y}$](#cell-sovling-lulike)
    0. [[PREREQUISITE] Gradients](#cell-sovling-gradients)

1. [Direct Methods finding](#cell-sovling-direct-methods) [$X\hat \beta \approx y \text{ by solving } \overset{A\, x}{X^TX \hat \beta} = \overset{b}{X^T y}$](#cell-sovling-direct-methods) (and see also `STA410_W24_Week5_Extra_MoreLeastSquares.ipynb`)

    1. [[OPTIONAL] "Big O" Algorithmic Complexity](#cell-sovling-direct-methods-big0)
    2. [Runtimes](#cell-sovling-runtimes)
        - [Comparing  `np.linalg.inv` and `np.linalg.solve`](#cell-sovling-runtimes-inv)
            - [Details on `np.linalg.solve` and `np.linalg.inv`](#cell-sovling-runtimes-solve)
        - [Alternative approaches with `np.linalg.qr`, `np.linalg.cholesky`, and `np.linalg.svd`](#cell-sovling-runtimes-others)
            - [Inversion with `np.linalg.svd`](#cell-sovling-runtimes-svdinv)

    3. [[OPTIONAL] Condition and Numerical Accuracy](#cell-sovling-direct-methods-considerations)



In [1]:
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib

<a name="cell-sovling-lulike"></a>

# 4.0 [OPTIONAL] Finding $X\hat \beta \approx y$ by solving for $\hat \beta$ in $\overset{A\, x}{X^T\!X \hat \beta} = \overset{b}{X^t y}$<br>([Return to TOC](#cell-solving))

---

## TLDR: $\hat \beta$ solving $\;X^TX\hat \beta = X^Ty\;$ provides the ***least squares*** approximation $X\hat \beta \approx  y$ 



The linear regression model $y = X\beta + \epsilon$ can be re-expressed as a so-called ($n>m$) ***overdetermined*** problem

$$\begin{align*}
 \overbrace{X_{n \times m} \hat \beta}^{A\,x} = {} & \overbrace{y - \hat \epsilon}^{b} 
\end{align*}$$

Of course unless $\epsilon=0$, this leads to the ***inconsistent*** system of equations $X \hat \beta \not = y$. However, $X \hat \beta \approx y$ if 

<!--
$$\begin{align*}
\hat \beta = {} & \underset{\quad \; \beta: \; \hat \epsilon \; = \; y - X\beta }{\text{argmin } ||\hat \epsilon||_2^2} \\
 = {} & \underset{\beta}{\text{argmin }} ||y - X \beta ||_2^2 = \underset{\beta}{\text{argmin }} (y - X \beta )^T(y - X \beta ) \\
\end{align*}
$$
-->

\begin{align*}
  \hat \beta = {} & \underset{\quad \; \beta: \; \hat \epsilon \; = \; X\beta - y}{\text{argmin } ||\hat \epsilon||_2^2} 
  = \underset{\beta}{\text{argmin }} ||y-X \beta||_2^2  \\
  = {} & \underset{\beta}{\text{argmin }} (y-X \beta)^T(y- X \beta) 
  = \underset{\beta}{\text{argmin }} \underbrace{\beta^TX^TX \beta -2\beta^TX^Ty}_{f(\beta)} \\
  0 = {} & \overbrace{\nabla_\beta \left( \beta^TX^TX \beta -2\beta^TX^Ty \right)}^{\nabla_\beta f(\hat \beta )}\\
  0 = {} &  X^TX \hat \beta - X^T y  \quad \Longrightarrow \quad  \underset{{\text{$\sum_i \hat \epsilon_i = 0$ if $X$ has an intercept column} }}{\overset{\text{residuals are othogonal to columns $X_{*j}$}}{0 = X^T (X \hat \beta - y) = X^T\hat \epsilon }} \\
  \Longrightarrow \quad X^T y = {} &  X^TX \hat \beta \quad \longleftarrow \quad \textrm{the so-called } \textit{normal equations}
  \end{align*}

so that $\hat \beta$ above is the solution to the ***normal equations*** $\overbrace{(X^TX) \hat \beta}^{A\,x} = \overbrace{X^TY}^{b}$, which leads to the familiar analytical solution $\underbrace{\hat \beta}_x = \underbrace{(X^TX)^{-1}}_{A^{-1}}\underbrace{X^TY}_b$.

So 

$$X \hat \beta\approx y \quad \text{ transforms into } \quad X^TX\hat \beta = X^Ty \quad (\text{or generically } Ax = b)$$ 

such that solving for $\hat \beta$ in the latter provides $X \hat \beta\approx y$.

<a name="cell-sovling-gradients"></a>

## 4.0.0 [PREREQUISITE] Gradients ([Return to TOC](#cell-solving))

--- 

The ***operator*** (i.e., function returning a function) $\nabla_{x}$ is called the ***gradient*** and generalizes the ***derivative for scalar quantities*** as the vector collection of all the ***partial derivatives*** of its argument, the ***scalar valued multivariate function*** $f_x$, with respect to input elements $x_i$ of $f_x$

$$ \nabla_{x}(f_x) = \frac{\partial f_x}{\partial x_1} e_1 + \cdots + \frac{\partial f_x}{\partial x_i} e_i + \cdots + \frac{\partial f_x}{\partial x_n} e_n $$

for ***standard basis vectors*** $e_i$.

Like the ***derivative***, the ***gradient*** is a ***linear operator*** with 
 
$$\nabla_{x}(af_x+bg_x) = a\nabla_{x}(f_x) + b\nabla_{x}(g_x)$$

and (as seen above) has the simple rules such as

$$\nabla_{x} x^Tb = b \quad \text{ and } \quad \nabla_{x} x^TAx = 2Ax$$

<a name="cell-sovling-direct-methods"></a>

## 4.1 Direct Methods for $X\hat \beta \approx y$ by solving for $\hat \beta$ in $\overset{A\, x}{X^TX \beta} = \overset{b}{X^T y}$ ([Return to TOC](#cell-solving))

---

***Direct methods*** execute a set sequence of predefined deterministic computational steps to arrive at the exact solution (to the limits of available precision) to a computational problem. The various `sum` and `variance` computations considered in the `STA410_W24_Week4_Homework_AdditionVariance.ipynb` are examples of ***direct methods***. Similarly, the ***Gram-Schmidt orthogonalization*** and ***Cholesky decomposition*** considered in the `STA410_W24_Week5_Extra_LinearAlgebraAlgorithms.ipynb` are examples of ***direct methods***. And the computation of the ***SVD*** is a ***direct method***. 
> While we have not seen exactly what the ***SVD*** algorithm is, suffice it to say that is is similar in character to the ***Gram-Schmidt orthogonalization*** and ***Cholesky decompositions***.

Here are some new examples of ***direct methods***

1. `np.linalg.inv(X.T@X) @ X^Ty # Analytical solution`
2. `np.linalg.solve(X.T@X, X^Ty) # Gaussian elimination`
3. `# QR decomposition based least squares estimates`

   `Q,R = np.linalg.qr(X); scipy.linalg.solve_triangular(R, Q.T@y)` 
   
   
4. `C = np.linalg.cholesky(X.T@X) # Cholesky decomposition based least squares estimates`

   `scipy.linalg.solve_triangular(C, scipy.linalg.solve_triangular(C, X.T@y, lower=True))`
   
   
5. `np.linalg.lstsq(X, y)  # SVD based least squares estimates` 

used to solve for the least squares solution $\hat \beta$ solving $X^TX\hat \beta = X^T y$ (or more generally for $x$ solving $Ax=b$ when a solution exists). These are ***direct methods*** because each each algorithmically computes the predifined steps of a linear algebra prescription that computates $\hat \beta$.

> All of these methods (for $X_{n\times n}$ with $n>m$) are $O(nm^2)$; although, importantly, they will have different computational run times for a given $n$ and $m$ since $M$ in $f(n,m)\leq Mnm^2$ can be (and is) different for each algorithm.

<a name="cell-sovling-direct-methods-big0"></a>

## [OPTIONAL] 4.1.0 "Big O" Algorithmic Complexity ([Return to TOC](#cell-solving))

---

> An algorithm requiring $f(n)$ operations to complete a problem with input size $n$ has ***Big $O$*** ***algorithmic complexity*** order $g(n)$, written as
> $$f(n) = O(g(n)) \quad \text{ if } \quad f(n) \leq M g(n) \quad \text{ for all $n>n_0$}$$
>
> for strictly positive $g$ and some constant $M$ that is not a function of $n$.
>
> - For further presentation of this topic please see `STA410_W24_Week5_Extra_SpeedAndBigOAlgorithmicComplexity.ipynb`


The "***Big O***" ***algorithmic complexity*** for $n>m$ of all of the ***direct methods*** above is $O(nm^2)$. 

- `np.linalg.inv(X.T@X) @ X^Ty` is $(X^T@X)^{-1} (X_{n\times m}^Ty) = O(nm^2)$ since this term dominates the operations together $O(nm^2) + O(m^3) + O(nm) + O(m^2)$ 
  - `X.T@X` matrix-matrix multiplication is $X_{n\times m}^TX_{n\times m} = O(nm^2)$ 
  - `np.linalg.inv(A_mxm)` [matrix inversion](http://gregorygundersen.com/blog/2020/12/09/matrix-inversion/) and even [***upper triangular matrix*** inversion](https://math.stackexchange.com/questions/92068/fast-inversion-of-a-triangular-matrix) is $X_{n\times m}^Ty = O(m^3)$ 
  - `X.T@y` matrix-vector multiplication is $X_{n\times m}^Ty = O(nm)$
  - `Ainv_mxm @ b_mx1` is $O(m^2)$.

- `np.linalg.solve(X.T@X, X.T@y)` is again $O(nm^2)$ as $X_{n\times m}^TX_{n\times m} = O(nm^2)$ will again dominate $X_{n\times m}^Ty = O(nm)$ and the other required $O(m^3)$ and $O(m^2)$ oprations
  - `np.linalg.solve(A_mxm, b)` is $O(m^3)$ since (as shown previously) it requires the sequential application of 

    1. ***Gaussian elimination*** with a required operation count of $2 \times \left(\frac{m(m+1)(2m+1)}{6} - m + \frac{m(mm+1)}{2} - m \right) = O(m^3)$ 

    2. ***backward substitution*** with a required operation count of $m^2 = O(m^2)$ 

- The ***QR decomposition*** `Q,R = np.linalg.qr(X)` using [***Householder transformations***](https://math.stackexchange.com/questions/501018/what-is-the-operation-count-for-qr-factorization-using-householder-transformatio) which are more numerically accurate than the ***(modified) Graham-Schmidt procedure*** is $O(nm^2)$ 
  - `scipy.linalg.solve_triangular(R, Q.T@y)` is again $m^2 = O(m^2)$ 

- The ***Cholesky decomposition*** `C = np.linalg.cholesky(X.T@X)` is a ***Gaussian elimination*** like process of $O(m^3)$, but the prerequesite $X_{n\times m}^TX_{n\times m}$ computation is $O(nm^2)$

  - `scipy.linalg.solve_triangular(C, scipy.linalg.solve_triangular(C, X.T@y, lower=True))` are each $m^2 = O(m^2)$ 

- The [standard computation](https://www.mathworks.com/matlabcentral/answers/420057-what-is-the-complexity-of-matlab-s-implementation-of-svd) of the ***SVD*** [implemented](https://scicomp.stackexchange.com/questions/6979/how-is-the-svd-of-a-matrix-computed-in-practice) by `np.linalg.lstsq(X, y)` is $O(nm^2)$

  - The subsquent $\hat \beta = V (D^{+})^2 V^T X^T y$ is $O(\max(m^3,nm))$ depending on if $VV^T$ or $X^T y$ is more computationally expensive




<a name="cell-sovling-runtimes"></a>

## 4.1.1 Runtimes ([Return to TOC](#cell-solving))

---

While all of the ***direct methods*** above are $O(nm^2)$, their actual operation counts differ proportionally, leading to proportionally different run times. 

<a name="cell-sovling-runtimes-inv"></a>

### Comparing  `np.linalg.inv` and `np.linalg.solve` ([Return to TOC](#cell-solving))

---

Ignoring `X.T@X` and `X.T@y` operations in $\overbrace{(X^TX)}^A\overset{x}{\beta} = \overbrace{X^Ty}^b$, `np.linalg.inv(A) @ b` is always proportionally slower than `np.linalg.solve(A, b)` regardless of $n$ in $A_{n\times n}$. So while  they have the same "***Big O***" ***algorithmic complexity*** of $O(nm^2)$, they have different computational requirements. This makes solving linear systems of equations using inverses computationally inefficient.

In [2]:
n = 500
A, b = stats.norm.rvs(size=(n,n)), stats.norm.rvs(size=(n,1))

In [3]:
%timeit np.linalg.inv(A) @ b

5.54 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%timeit np.linalg.solve(A, b)

2.96 ms ± 70.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
n = 1000
A, b = stats.norm.rvs(size=(n,n)), stats.norm.rvs(size=(n,1))

In [6]:
%timeit np.linalg.inv(A) @ b

21.5 ms ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%timeit np.linalg.solve(A, b)

10.7 ms ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
n = 2000
A, b = stats.norm.rvs(size=(n,n)), stats.norm.rvs(size=(n,1))

In [9]:
%timeit np.linalg.inv(A) @ b

112 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit np.linalg.solve(A, b)

49.8 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


<a name="cell-sovling-runtimes-solve"></a>

### Details on `np.linalg.solve` and `np.linalg.inv` ([Return to TOC](#cell-solving))

---

> For more background on the topics of this section please see `STA410_W24_Week4_Prerequesite_LinearAlgebra.ipynb`. 



The combined ***Gaussian elimination*** and ***backward substitution*** of `np.linalg.solve` is mathematically stated in terms of the ***LU decomposition*** as $Ux = L^{-1}b$. However, the actual ***LU decomposition***


$$A|I|b \quad \underset{\text{Elimination}}{\overset{\text{Gaussian}}{\longrightarrow}} \quad \overbrace{U|L^{-1}|b'}^{\text{LU decompostion}}$$

which can be computed with `P,L,U = scipy.linalg.lu(A)` is not necessary; rather, a computationally efficient representation `LU = scipy.linalg.lu_factor(A)` can be created and subsequently used to solve for $x$ using `scipy.linalg.lu_solve(LU, b)`. 

This means that the actual computation of $A^{-1}$ is not done by a continuation of $O(n^3)$ ***Gaussian elimination*** as

$$A|I|b \quad \underset{\text{Elimination}}{\overset{\text{Gaussian}}{\longrightarrow}} \quad \overbrace{U|L^{-1}|b'}^{\text{LU decompostion}}  \quad \overset{\text{more}}{\underset{\text{Elimination}}{\overset{\text{Gaussian}}{\longrightarrow}}} \quad I|A^{-1}|b''$$

but instead by just solving for each column of $A^{-1}$ as

$$UA^{-1}_{\cdot j} = L^{-1}e_j \text{ for each } j$$

with `scipy.linalg.lu_solve(LU, b)` once the initial efficient representation of the ***LU decomposition*** `LU = scipy.linalg.lu_factor(A)` is obtained.

The `scipy.linalg.lu_solve(LU, b)` is not performing $O(n^2)$ ***Gaussian elimination*** for each column $A^{-1}_{\cdot j}$ though; rather, it is an $O(n)$ algorithm that is based on the efficient `LU` representation.

> Solving for $x$ in $Ax=b$ means solving just a single system of linear equations. But finding $A^{-1}$ by solving $UA^{-1}_{\cdot j} = L^{-1}e_j$ for $j=1,\cdots,n$ sugguests that to compute the inverse requires solving $n$ systems of linear equations. So why is computing $A^{-1}$ not $n$ times slower than solving a system of linear equations? 
>
> This is because even though the $O(n^3)$ `LU=scipy.linalg.lu_factor(A)` computation is proportionally slower than the $O(n^3)$ `np.linalg.solve(A,b)`, the subsequent $n$ calls to $O(n)$ `scipy.linalg.lu_solve(LU, b)` means that computing the columns of $A^{-1}$ after $O(n^3)$ `LU=scipy.linalg.lu_factor(A)` has completed is only $O(n^2)$, which is an insignificant computational requirement compared to $O(n^3)$ when $n$ is large.


In [11]:
n = 1000
A, b = stats.norm.rvs(size=(n,n)), stats.norm.rvs(size=(n,1))

In [12]:
%timeit np.linalg.solve(A, b)

10.8 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
# what `np.linalg.solve(A, b)` above is doing
%timeit LU = scipy.linalg.lu_factor(A); scipy.linalg.lu_solve(LU, b)

11 ms ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
# `scipy.linalg.lu_solve(LU, b)` above barely matters!
%timeit LU = scipy.linalg.lu_factor(A)
# because O(n^2) scales so much more reasonably than O(n^3) as n grows...

10.5 ms ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
LU = scipy.linalg.lu_factor(A)

In [17]:
# obviously if we have LU precomputed, then we can (and should just) resue it on each column
%timeit scipy.linalg.lu_solve(LU, np.eye(n))
# This does the O(n^3) task once, then uses the result for the O(n^2) task

12.7 ms ± 379 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
# that's what this is: scipy.linalg.lu_factor(A) then scipy.linalg.lu_solve(LU, np.eye(n))
%timeit np.linalg.inv(A)

23.9 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
# here `scipy.linalg.lu_factor(A)` is being recomputed each time...
%timeit [np.linalg.solve(A, np.eye(n)[:,i]) for i in range(int(n/25))]
# This does the O(n^3) task over, and over, and over, ... super wasteful

455 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
# LU from `lu_factor(A)` above just stores things so all the computations needed
# to actually do `lu_solve(LU, b)` can be readily and most efficiently completed
# AKA: we don't actually need to produce L and U of the LU decomposition

In [21]:
# to actually construct L and U takes more time
%timeit P,L,U = scipy.linalg.lu(A)
# P is the row/col interchanges, i.e., the permutation matrix

15.8 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# Similarly, (1) to produce L-inverse will be computationally expensive; and, then
# (2) to L^-1 @ b will be computationally expensive; and, only then can we finally
# (3) use the upper triangular U to backward substitution solve for x against L^-1 @ b
# https://math.stackexchange.com/questions/1009916/easy-way-to-calculate-inverse-of-an-lu-decomposition
# ...
# So, the `lu_factor(A)` and the `lu_solve(LU, b)` sequence is the way to go
# without actually building L, U, and L^-1, and all that other related stuff
# hashtag "algorithmic choices"

<a name="cell-sovling-runtimes-others"></a>

### Alternative approaches with `np.linalg.qr`, `np.linalg.cholesky`, and `np.linalg.svd` ([Return to TOC](#cell-solving))
---

The ***QR decomposition*** approach is again proportionally slower than both `np.linalg.solve(X.T@X, X.T@y)` and `np.linalg.inv(X.T@X)@(X.T@y)`. 

- While `np.linalg.solve` is proportionally faster than `np.linalg.inv`, both run times here are dominated by the ***gramian*** computation `X.T@X` so this efficiency benefit is minor in light of the total computational expense. The ***QR decomposition*** doesn't compute `X.T@X` but the factorization itself into `QR` is $O(n^2m)$ like `X.T@X` (but proportionally slower overall). 

- The benefit of `np.linalg.solve` being proportionally faster thant `np.linalg.inv` would be relevant for smaller $n$ in $X_{n\times m}$ where the cost of the ***gramian*** computation `X.T@X` was less dominant in the overall run time. Both `np.linalg.solve` and `np.linalg.inv` are $O(m^3)$ algorithms, so even though `np.linalg.solve` is proportionally faster, the $O(nm^2)$ ***gramian*** computation `X.T@X` dominates the $O(m^3)$ costs when $n>> m$.

The ***Cholesky decomposition*** approach will be about the same speed as  `np.linalg.solve(X.T@X, X.T@y)`; but, again, the computation times are still dominated by the ***gramian*** computation `X.T@X` for large $n$ so any computational speed differences between `np.linalg.cholesky` and  `np.linalg.solve` will be minor in light of the total computational expense.

- Of course, `np.linalg.cholesky` has half the operations and space requirements of `scipy.linalg.lu` which represents the dominant (***Gaussian elimination*** like) computation in `np.linalg.solve`; however, the algorithm underlying `scipy.linalg.lu` can be vectorized while the algorithm underlying `np.linalg.cholesky` cannot. For $X_{n \times m}$ with relatively large $m$, `np.linalg.cholesky` is notably faster than `scipy.linalg.lu`; but, for smaller $m$ the opposite is true as the speed benefits of the vectorization `scipy.linalg.lu` overtake the reduced operation and space requirements of `np.linalg.cholesky`. 

> Have a look at the nature of ***Gaussian elimination*** given in  `STA410_W24_Week4_Prerequesite_LinearAlgebra.ipynb` versus the ***Cholesky decomposition*** from 
`STA410_W24_Week5_Extra_LinearAlgebraAlgorithms.ipynb` to get a since of why the former algorithm can be "vectorized" while the latter cannot. 


The ***SVD*** approach is again proportionally slower than the ***QR decomposition*** approach; although, for $n>>m$ the run times of these two algorithms become increasingly similar. 

#### QR Decomposition
---

In [22]:
n,m = 10000,1000
X, y = stats.norm.rvs(size=(n,m)), stats.norm.rvs(size=(n,1))
Q,R = np.linalg.qr(X)

In [23]:
%timeit Q,R = np.linalg.qr(X); scipy.linalg.solve_triangular(R, Q.T@y)

1.43 s ± 199 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
# insignificant
%timeit scipy.linalg.solve_triangular(R, Q.T@y)

3.38 ms ± 32.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%timeit np.linalg.solve(X.T@X, X.T@y)

85.4 ms ± 2.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
# wasteful / unnecessary 
%timeit np.linalg.inv(X.T@X)@(X.T@y)
# increased computation -> increased roundoff error

96.5 ms ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit X.T@X

70.4 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Cholesky Decomposition
---

In [28]:
S,b = X.T@X, X.T@y # if timing these computations is not desired
C = np.linalg.cholesky(S)

In [29]:
%timeit C = np.linalg.cholesky(X.T@X); scipy.linalg.solve_triangular(C.T, scipy.linalg.solve_triangular(C, b, lower=True))
# about the same speed as np.linalg.solve(X.T@X, X.T@y) above

93.9 ms ± 8.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
# insignificant
%timeit scipy.linalg.solve_triangular(C.T, scipy.linalg.solve_triangular(C, b, lower=True))

559 µs ± 8.42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
# ignoring the grammian X.T@X computation
%timeit C = np.linalg.cholesky(S); scipy.linalg.solve_triangular(C.T, scipy.linalg.solve_triangular(C, b, lower=True))

14.3 ms ± 491 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
# the above doesn't actually compute full LU decomposition,
# which uses twice as many computations (which should be twice as slow...)
# But the LU decomposition benefits from vectorization
# not available in the Cholesky decomposition
%timeit np.linalg.solve(S, b)

10.2 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%timeit scipy.linalg.lu(S) # LU=u_factor.scipy.linalg(S); scipy.linalg.lu_solve(LU, b)

12.7 ms ± 315 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### SVD
---

In [46]:
n,m = 1000,100
X,y = stats.norm.rvs(size=(n,m)), stats.norm.rvs(size=(n,1))
Q,R = np.linalg.qr(X) 
S,b = X.T@X, X.T@y # if timing these computations is not desired
C = np.linalg.cholesky(S)

In [47]:
%timeit C = np.linalg.cholesky(X.T@X); scipy.linalg.solve_triangular(C.T, scipy.linalg.solve_triangular(C, X.T@y, lower=True))

884 µs ± 56.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [48]:
%timeit np.linalg.solve(X.T@X, X.T@y)

585 µs ± 7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [49]:
%timeit np.linalg.inv(X.T@X)@(X.T@y)

718 µs ± 5.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [50]:
%timeit Q,R = np.linalg.qr(X); scipy.linalg.solve_triangular(R, Q.T@y)
# takes time to store QR

18.1 ms ± 68.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [51]:
# SVD version
%timeit np.linalg.lstsq(X, y, rcond=None)
# https://stats.stackexchange.com/questions/240573/how-does-numpy-solve-least-squares-for-underdetermined-systems
# https://math.stackexchange.com/questions/3252351/when-solving-a-linear-system-why-svd-is-preferred-over-qr-to-make-the-solution

26 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
%timeit X.T@X

186 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


<a name="cell-sovling-runtimes-svdinv"></a>

### Inversion with `np.linalg.svd` ([Return to TOC](#cell-solving))
---

As seen below, calculating 
$A^{-1} = VD^{-1}U^T$ from the ***orthogonality*** of the ***SVD*** (or ***Eigendecomposition***) $A = UDV^T$ using `np.linalg.svd(A)` along with the subsequent $VD^{-1}U^T$ computation is many times slower than `np.linalg.inv(A)`, so the analytical inverse available from ***SVD*** does not actually represent any sort of computational shortcut relative to calculating $A^{-1}$.

In [53]:
n = 1000
A = stats.norm.rvs(size=(n,n))
U,D,Vt = np.linalg.svd(A)

In [54]:
%timeit np.linalg.inv(A)

22.9 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
# `np.linalg.svd` is a more expensive O(n^3) operation than `np.linalg.inv()`
%timeit U,D,Vt = np.linalg.svd(A); Vt.T @ U.T / D

703 ms ± 57.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
# even if the O(n^3) matrix multiplication would be pretty fast
%timeit Vt.T @ U.T / D

14 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


<a name="cell-sovling-direct-methods-considerations"></a>

## 4.1.2 [OPTIONAL] Condition and Numerical Accuracy ([Return to TOC](#cell-solving))

---

Since $ \hat \beta = (X^T\!X)^{-1}X^T\!y$, the ***condition*** of the problem $X^T\!X \beta = X^T\!y$ is $\kappa(X^TX) = \frac{\lambda^{\max}_{X^TX}}{\lambda^{\min}_{X^TX}}$ regardless of how the problem is solved. So if $\kappa(X^TX)$ is very large then the problem itself is ***ill-conditioned***. 
- Thus, in the context of an ***ill-conditioned*** problem, even the most minor ***roundoff error*** can lead to wildly inaccurate solutions, and so even the most accurate computations can still fail miserably for ***ill-conditioned*** problems. 

Nonetheless, the different ***direct method*** algorithms for finding the ***least squares*** solution to a ***linear regression*** problem do indeed actually perform different computations and factorizations, and so when there is low ***multicollinearity*** in $X$ (and the ***singular values*** are similarly scaled and $X$ is ***well-conditioned***), the different methods have different amounts of ***roundoff error***. However, none of the methods really incur prohibitive ***roundoff error*** when $X$ is ***well-conditioned***.

|                      | Grammian Computations       | Actual Computational Requirements | ***Roundoff Error*** Considerations for Small $\kappa(X)$ | and for Very Large $\kappa(X)$ |
|----------------------|-----------------------------|---------------------|----|---|
| `np.linalg.inv`      | ✘ $\kappa(X^TX)$ | $O(nm^2) \; + \underset{\text{[vectorizable]}}{O(m^3)} +  O(m^2)$ | $X^TX$ computation will be sufficiently accurate<br>computations beyond $X^TX$ add ***roundoff error*** | ***Ill-Conditioned Problem***| 
| `np.linalg.solve` | ✘ $\kappa(X^TX)$ | $O(nm^2) + \underset{\text{[vectorizable]}}{O(m^3)} +  O(m)$ | $X^TX$ computation will be sufficiently accurate<br>requires $O(m)$ not $O(m^2)$ last stage operations | ***Ill-Conditioned Problem***|
| `np.linalg.qr`       |  $\checkmark \;\kappa(X)$  | $\underset{>>1}{a}\times O(nm^2)$ | ***QR*** is often a little bit more accurate than $X^TX$ | ***Ill-Conditioned Problem***|
| `np.linalg.cholesky` | ✘ $\kappa(X^TX)$ | $O(nm^2) + \sim \frac{1}{2}\times O(m^3)$ | As good or better than using `np.linalg.solve` | ***Ill-Conditioned Problem***|
| `np.linalg.svd`      |  $\checkmark \;\kappa(X)$              | $\underset{>>a}{b}\times O(nm^2)$ | ***SVD*** is typically thought to be the most accurate | ***Ill-Conditioned Problem***|

<!--
1. complete computation of $\hat \beta = (X^T\!X)^{-1} X^Ty$
2. ***Gaussian elimination*** of the ***gramian*** $X^TX$ and $X^T\!y$ followed by ***backward substitution*** to solve $\hat \beta$
3. ***QR decomposition*** of $X$ followed by ***backward substitution*** with $R$ and $Q^Ty$ to solve for $\hat \beta$
4. ***Cholesky decomposition*** of the (***symmetric positive definite***) ***gramian*** $X^TX=CC^T$ followed by two rounds of ***backward substitution*** based on $C$ and $C^T$ to subsequently solve for $\hat \beta$
5. ***SVD*** of $X$ and computation of $\hat \beta = V (D^{+})^2 V^T X^T y$
-->



- `np.linalg.inv(X.T@X) @ X^Ty`

  > If $X$ is ***full rank*** then $(X^TX)^{-1}$ exists; however, if columns of $X$ are highly ***multicollinear*** the ***condition number*** $\kappa(X^TX)$ will be large and the problem will be ***ill-conditioned*** so that ***roundoff error*** in $X^Ty$ and $(X^TX)^{-1}$ may lead to a solution $\hat \beta = (X^TX)^{-1}X^Ty$ which is very inaccurate.
  > - For a ***well-conditioned*** problem with small $\kappa(X^TX)$, this solution is *still not recommended* because it requires unnecessary computations which can add ***roundoff error***.


- `np.linalg.solve(X.T@X, X^Ty)` 

  > Rather than unnecessarily computing $(X^TX)^{-1}$ for ***full rank*** $X$ which is both computationally wasteful and can add additional ***roundoff error***, ***Gaussian Elimination*** and ***backward substitution*** can be used to solve for $\hat \beta$ in $X^TX \hat \beta = X^T Y$. 
  > - For a ***well-conditioned*** problem with small $\kappa(X^TX)$ this solution will be perfectly numerically satisfactory and result in negligable ***roundoff error***.

- `Q,R = np.linalg.qr(X); scipy.linalg.solve_triangular(R, Q.T@y)`

  > If $X$ is ***full rank*** then ***QR decomposition*** $X=QR$ transforms $X^TX \hat \beta = X^T Y$ to $R \hat \beta = Q^T Y$ where $R$ is already a ***triangular matrix*** so $\hat \beta$ can be immediately solved for ***backward substitution***.
  > - If $\kappa(X)$ is small then $Q$ and $R$ produced by the ***QR decomposition*** will have minimal ***roundoff error***, and subsequently the ***condition numbers*** $\kappa(X) = \kappa(R) < \kappa(R^TR) = \kappa(X^TX)$ for solving for $\hat \beta$ in $R \hat \beta = Q^T Y$
  and $\kappa(Q)=\kappa(Q^T)=1$ for computing $Q^T y$, all of which leads to a typically reduced ***roundoff error*** compared to computing $(X^TX)$ and solving for $\hat \beta$ in $(X^TX)\hat \beta = X^Ty$.
  > - Regardless, since $\hat \beta = R^{-1}Q^T y = (X^TX)^{-1}X^Ty$ is the answer to $f_{R^{-1}Q^T}(y) = f_{(X^TX)^{-1}X^T}(y)$, even when $y+\epsilon \approx y$ so that
  >   - $Q^T (y+\epsilon)$ is expected to have less ***roundoff error*** than $X^T(y+\epsilon)$
  >   - solving for $\hat \beta$ in $R\hat \beta = Q^T (y+\epsilon)$ is also expected to have less ***roundoff error*** than solving for $\hat \beta$ in $X^T\!X\hat \beta = X^T (y+\epsilon)$
  > 
  >   it's still not the case that $\hat \beta_y \overset{?}{\approx} \hat \beta_{y+\epsilon} = f_{R^{-1}Q^T}(y+\epsilon) = f_{(X^TX)^{-1}X^T}(y+\epsilon)$ when the problem is ***ill-conditioned*** to begin with. The reduced ***roundoff error*** from the use of the ***QR decomposition*** will not overcome problem ***ill-conditioning***
  >   - not to mention (equivalently) that this will be a problem suffering from high multicollinearity which means the statistical analysis will be generally problematic to start with; but, it is true that we would expect the point estimate might prove more accurate with the ***QR decomposition*** if that's all you were looking for...  
 
- `C = np.linalg.cholesky(X.T@X); scipy.linalg.solve_triangular(C, scipy.linalg.solve_triangular(C, X.T@y, lower=True))`

  > For ***full rank*** $X$, the ***Cholesky decomposition*** can be substituted in place of ***Gaussian elimination*** and then $\hat \beta$ in 
  >
  > $$X^T\!X\hat \beta = CC^T \hat \beta =  \underset{L}{C} (\underbrace{C^T \hat \beta}_{\gamma}) = X^T\!y$$ 
  > can be found by solving two ***backward substitution*** problems. 
  >
  > - Following computation of $X^TX$, the choice of ***Gaussian elimination*** or ***Cholesky decomposition*** does not greatly impact numerical accuracy. Still, ***backward substitution*** solving for $\gamma$ in $C \gamma = X^T\!y $ and then $\hat \beta$ in $C \hat \beta = \gamma$ actually does have a reduced ***condition number*** $\kappa(C) < \kappa(X^TX)$, so the ***Cholesky decomposition*** approach is a perfectly viable alternative to the other methods for ***well-conditioned*** problems with small $\kappa(X^TX)$. 
  > - For the same reasons as discussed in the context of a ***QR decomposition***, the problem ***condition*** of $\hat \beta = (CC^T)^{-1}X^Ty = (X^TX)^{-1}X^Ty$ is not a function of whether or not the ***Cholesky decomposition*** is used; but, interestingly, even if $\text{rank}(X)=p$, when $\kappa(X^T\!X)$ is large the problem ***ill-conditioning*** is immediately obvious because the ***symmetric*** $X^T\!X = \frac{X^T\!X +(X^T\!X)^T}{2}$ will not be ***positive definite*** and substantial reconstruction ***roundoff error*** will be observed in $CC^T$.
  >   - No method has the ability to overcome solution volatility caused by ***roundoff error*** in the context of ***ill-conditioned*** problem; but, the problem ***ill-conditioned*** can be numerically diagnosed from the ***Cholesky decomposition*** itself.

- `np.linalg.lstsq(X, y)`

  > For the ***SVD*** of any matrix $X = UDV^T$ the ***Moore-Penrose (generalized) inverse*** $(X^TX)^+ = V (D^{+})^2 V^T \underset{\text{full rank}}{\overset{\text{if $X$ is}}{=}} (X^TX)^{-1}$ with $(D^{+}_{ii})^2=\frac{1}{D_{ii}^2}$ specifies $\hat \beta=V (D^{+})^2 V^T X^Ty$ so that only the ***SVD*** of $X$ and some final matrix multiplications are needed for this approach.
  > - Although $\kappa((D^{+}_{ii})^2) = \kappa(X^TX)$ the ***roundoff error*** in $(D^{+}_{ii})^2$ is generally less than in $X^TX$, and $V$ similarly tends to have less ***roundoff error*** than $Q$ and $R$ of the ***QR decomposition***, all of which leads to the ***SVD*** approach generally being the most accurate of all the approaches.
  >   - The increased computational costs of the ***SVD*** approach, then, do not represent unnecessary computations, but additional computations which have the affect of minimizing ***roundoff error*** during the decomposition. 